In [ ]:
import telebot
import requests
import jsons
from Class_ModelResponse import ModelResponse

# Замените 'YOUR_BOT_TOKEN' на ваш токен от BotFather
API_TOKEN = 'YOUR_BOT_TOKEN'
bot = telebot.TeleBot(API_TOKEN)

# Хранилище контекста для каждого пользователя
user_contexts = {}

# Команды
@bot.message_handler(commands=['start'])
def send_welcome(message):
    user_id = message.from_user.id
   
    welcome_text = (
        "👋 Привет! Я ваш цифровой помощник.\n"
        "Используйте /help для просмотра всех доступных команд.\n"
    )
    bot.reply_to(message, welcome_text)

@bot.message_handler(commands=['help'])
def send_help(message):
    help_text = (
        "📚 Доступные команды:\n\n"
        "/start - Начало работы\n"
        "/model - Показать используемую языковую модель\n"
        "/clear - Очистить контекст текущего диалога\n"
        "/help - Показать это сообщение с командами"
    )
    bot.reply_to(message, help_text)

@bot.message_handler(commands=['model'])
def send_model_name(message):
    try:
        response = requests.get('http://localhost:1234/v1/models')
        if response.status_code == 200:
            model_info = response.json()
            model_name = model_info['data'][0]['id']
            bot.reply_to(message, f"🧠 Используемая модель: {model_name}")
        else:
            bot.reply_to(message, 'Не удалось получить информацию о модели. Сервер вернул код ошибки: ' + str(response.status_code))
    except Exception as e:
        bot.reply_to(message, f'Ошибка подключения к серверу модели: {str(e)}')

@bot.message_handler(commands=['clear'])
def clear_context(message):
    user_id = message.from_user.id
    if user_id in user_contexts:
        del user_contexts[user_id]
    bot.reply_to(message, "🧹 Контекст успешно очищен!")

@bot.message_handler(func=lambda message: True)
def handle_message(message):
    user_id = message.from_user.id
    user_query = message.text
    
    if user_id not in user_contexts:
        user_contexts[user_id] = []
    
    user_contexts[user_id].append({"role": "user", "content": user_query})
    
    request = {
        "messages": user_contexts[user_id]
    }
    
    try:
        response = requests.post(
            'http://localhost:1234/v1/chat/completions',
            json=request,
            timeout=300  # Таймаут 300 секунд для запроса
        )

        if response.status_code == 200:
            model_response: ModelResponse = jsons.loads(response.text, ModelResponse)
            assistant_response = model_response.choices[0].message.content
            
            user_contexts[user_id].append({
                "role": "assistant",
                "content": assistant_response
            })
            
            bot.reply_to(message, assistant_response)
        else:
            bot.reply_to(message, f'Ошибка API модели (код {response.status_code})')

    except Exception as e:
        bot.reply_to(message, f'Произошла ошибка при обращении к модели: {str(e)}')

# Запуск бота
if __name__ == '__main__':
    bot.polling(none_stop=True)